# Hydroclimatic hazard - Irrigation filtration 
- Author: Eunkyoung Choi (kyoung.choi@colostate.edu)
- Version: June, 2021

In [11]:
### import packages
import numpy as np
import pandas as pd
import scipy.io
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# 1) Option

In [2]:
'''
Please select which one to model:
crop_name = Maize, Soybeans, Spring Wheat, Sorghum
crop_yld_name = maize_yld, soy_yld, spr_wheat_yld, sorghum_yld
'''

crop_name = 'Maize'
crop_yld_name = 'maize_yld'

first_yr = 1981
last_yr = 2020

In [3]:
############################## Custom Functions #############################################################################
def usda_format(usda_crop,val_name, nass):
    usda_crop = usda_crop.rename(columns={'Value':val_name,'State ANSI':'state_id','County ANSI':'county_id','Year':'year'})
    usda_crop['state_id'] = usda_crop['state_id'].astype(str).str.zfill(2)
    usda_crop = usda_crop.loc[~usda_crop['County'].isin(['OTHER (COMBINED) COUNTIES','OTHER COUNTIES'])]
    usda_crop = usda_crop.loc[~usda_crop['county_id'].isnull()]
    usda_crop['county_id'] = usda_crop['county_id'].astype(int).astype(str).str.zfill(3)
    usda_crop['GEOID'] = usda_crop['state_id'] + usda_crop['county_id']
    if nass == 'Yield':
        usda_crop = usda_crop.loc[usda_crop['county_id'] != '998'].copy()
    elif nass == 'Harvested':    
        usda_crop.loc[usda_crop[val_name] == ' (D)', val_name] = np.nan
        usda_crop[val_name] = usda_crop[val_name].str.replace(',','').astype(float)
    return usda_crop[['State','state_id','county_id','GEOID','year',val_name,'Data Item']]

def geoid_format(data):
    data['GEOID'] = data['GEOID'].astype(str).str.zfill(5)
    data['county_id'] = data['county_id'].astype(str).str.zfill(3)
    data['state_id'] = data['state_id'].astype(str).str.zfill(2)
    print(data.shape)
    return data.loc[(data['year'] > first_yr-1) & (data['year']<last_yr+1)]

# 1) Import data

In [5]:
## lon lat coordinates from USDA (USE 2016 DATA for PRISM and if GEOID is available, then I can use 2017 data for CONUS)
lonlat = pd.read_csv('USDA_2017_County_Boundary_Reprojected_EPSG4326.csv', index_col=[0])
lonlat['GEOID'] = lonlat['GEOID'].astype(str).str.zfill(5)
lonlat.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,centroid,lon,lat
0,39,131,1074078,0500000US39131,39131,Pike,6,1140324458,9567612,"POLYGON ((-83.35353099999999 39.197585, -82.78...",POINT (-83.06769584018872 39.07634000872211),-83.067696,39.076340
1,46,3,1266983,0500000US46003,46003,Aurora,6,1834813753,11201379,"POLYGON ((-98.807771 43.935223, -98.331508 43....",POINT (-98.5605057741495 43.71757615992276),-98.560506,43.717576
2,55,35,1581077,0500000US55035,55035,Eau Claire,6,1652211310,18848512,"POLYGON ((-91.65045499999999 44.855951, -90.92...",POINT (-91.28609183386752 44.72661400163859),-91.286092,44.726614
3,72,145,1804553,0500000US72145,72145,Vega Baja,13,118766803,57805868,"POLYGON ((-66.448989 18.387214, -66.4389500073...",POINT (-66.39840061996379 18.42641444057773),-66.398401,18.426414
4,48,259,1383915,0500000US48259,48259,Kendall,6,1715747531,1496797,"POLYGON ((-98.920147 30.13829, -98.587897 30.1...",POINT (-98.71135955694437 29.94566143682303),-98.711360,29.945661


## 1-1) Yield data

In [6]:
if crop_yld_name == 'maize_yld':
    data_path = '1_corn/1_survey_field_crop_total_yield_USDA/'
    usda_data = pd.read_csv(data_path+'survey_total_yield_maize_1981_2020_USDA_final_28Mar2022.csv', index_col=[0])
elif crop_yld_name == 'soy_yld':
    data_path = '2_soybeans/1_survey_field_crop_soybean_total_yields/'
    usda_data = pd.read_csv(data_path+'survey_total_yield_soybeans_1981_2020_USDA_final_12Apr2022.csv', index_col=[0])
elif crop_yld_name == 'sorghum_yld':
    data_path = '4_sorghum/1_survey_field_crop_sorghum_total_yields/'
    usda_data = pd.read_csv(data_path+'survey_total_yield_sorghum_1981_2020_USDA_final_24feb2022.csv', index_col=[0])
elif crop_yld_name == 'spr_wheat_yld':
    data_path = '3_spring_wheat/1_survey_field_crop_spring_wheat_total_yields/'
    usda_data = pd.read_csv(data_path+'survey_total_yield_spring_wheat_1981_2020_USDA_final_24feb2022.csv', index_col=[0])

usda_data = geoid_format(usda_data)
print(usda_data['Data Item'].unique())

(78010, 9)
['CORN, GRAIN - YIELD, MEASURED IN BU / ACRE']


In [7]:
## merging it with lonlat for plotting later:
usda_data = pd.merge(usda_data, lonlat[['GEOID','lon','lat']], on=['GEOID'], how='outer', indicator=True)
usda_data.loc[usda_data._merge == 'left_only']


,State_name,state_id,county_id,GEOID,year,durum_yld,Data Item,State,lon,lat,_merge


In [8]:
print(usda_data._merge.unique()) #left only here indicates old counties which do not exist in USDA now
usda_data = usda_data.loc[usda_data._merge == 'both'].drop(columns=['_merge']) 

['both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']


In [9]:
## check zero or nana value:
print(usda_data.shape)
print(usda_data.loc[usda_data[crop_yld_name] == 0.0].shape)
usda_data.loc[usda_data[crop_yld_name].isnull()].shape

(3185, 10)
(0, 10)


(7, 10)

In [10]:
# there is no 29193 county ? for maize and soybeans: the following county is gone in lonlat dataset.
usda_data.loc[usda_data['GEOID'] == '29193']
#data0.head()

,Program,State_name,state_id,county_id,GEOID,year,maize_yld,Data Item,State,lon,lat
41647,SURVEY,MISSOURI,29,193,29193,1986.0,95.9,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN
41648,SURVEY,MISSOURI,29,193,29193,1985.0,98.0,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN
41649,SURVEY,MISSOURI,29,193,29193,1984.0,85.2,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN
41650,SURVEY,MISSOURI,29,193,29193,1983.0,54.4,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN
41651,SURVEY,MISSOURI,29,193,29193,1982.0,103.0,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN
41652,SURVEY,MISSOURI,29,193,29193,1981.0,106.8,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",MO,NaN,NaN


## 1-2) Harvest data

In [9]:
if crop_yld_name == 'maize_yld':
    data_path = '1_corn/2_census_field_crop_harvested_acrea_USDA/'
    census_data = pd.read_csv(data_path+'census_maize_combined_total_irrigated_harvest_counties_28Mar2022.csv', index_col=[0])
elif crop_yld_name == 'soy_yld':
    data_path = '2_soybean/2_cencus_field_crop_soybean_acres/'
    census_data = pd.read_csv(data_path+'census_soybeans_combined_total_irrigated_harvest_counties_12Apr2022.csv', index_col=[0])
elif crop_yld_name == 'sorghum_yld':
    data_path = '4_sorghum/2_census_field_crop_sorghum_acres/'
    census_data = pd.read_csv(data_path+'census_sorghum_combined_total_irrigated_harvest_counties_12Apr2022.csv', index_col=[0])
elif crop_yld_name == 'spr_wheat_yld':
    data_path = '3_spring_wheat/2_census_field_crop_spring_wheat_acres/'
    census_data = pd.read_csv(data_path+'census_spring_wheat_combined_total_irrigated_harvest_counties_12Apr2022.csv', index_col=[0])

census_data = geoid_format(census_data)
census_data.head()

(11629, 10)


,State,state_id,county_id,GEOID,year,tot_harvest,irri_harvest,tot+irri_har_merge,lon,lat
0,ALABAMA,01,001,01001,2017.0,645.0,NaN,left_only,-86.643648,32.538666
1,ALABAMA,01,001,01001,2012.0,525.0,NaN,left_only,-86.643648,32.538666
2,ALABAMA,01,001,01001,2007.0,1464.0,NaN,left_only,-86.643648,32.538666
3,ALABAMA,01,001,01001,2002.0,1709.0,NaN,left_only,-86.643648,32.538666
4,ALABAMA,01,001,01001,1997.0,1445.0,NaN,left_only,-86.643648,32.538666


# 2) Irrigated county filtration
- Exclude the counties that reported only for the year 2017 (But counties that have one year report prior to 2017 will still be used, as that is only their report representing for 1981-2020.) 
- Get the counties with its averaged yearly irrigation fraction > 10% (This threshold came from the sensitivity test of corn (Bulter et al., 2018 PNAS & 2013 Nature). This should be re-examed if other crops have different sensitivities though.

In [12]:
harvest = census_data.copy()
#unit simplification
harvest['tot_harvest'] = harvest['tot_harvest'] * 1e-3
harvest['irri_harvest'] = harvest['irri_harvest'] * 1e-3
harvest.loc[(harvest['tot_harvest'].isnull()) | (harvest['irri_harvest'].isnull())]

,State,state_id,county_id,GEOID,year,tot_harvest,irri_harvest,tot+irri_har_merge,lon,lat
0,ALABAMA,01,001,01001,2017.0,0.645,NaN,left_only,-86.643648,32.538666
1,ALABAMA,01,001,01001,2012.0,0.525,NaN,left_only,-86.643648,32.538666
2,ALABAMA,01,001,01001,2007.0,1.464,NaN,left_only,-86.643648,32.538666
3,ALABAMA,01,001,01001,2002.0,1.709,NaN,left_only,-86.643648,32.538666
4,ALABAMA,01,001,01001,1997.0,1.445,NaN,left_only,-86.643648,32.538666
...,...,...,...,...,...,...,...,...,...,...
11630,VIRGINIA,51,027,51027,1997.0,0.013,NaN,left_only,-82.042009,37.265030
11631,VIRGINIA,51,161,51161,1997.0,0.064,NaN,left_only,-80.064062,37.269306
11632,WASHINGTON,53,013,53013,1997.0,0.068,NaN,left_only,-117.905198,46.294429
11633,WEST VIRGINIA,54,033,54033,1997.0,0.036,NaN,left_only,-80.379196,39.285385


In [16]:
harvest.loc[(harvest['tot_harvest'] == 0.0) | (harvest['irri_harvest'] ==0.0)]

,State,state_id,county_id,GEOID,year,tot_harvest,irri_harvest,tot+irri_har_merge,lon,lat,yr_irri_frac


In [2]:
### plotting the spatial map of total and irrigated harvested areas:
temp_df = harvest.groupby(['GEOID'])[['tot_harvest','irri_harvest']].mean().reset_index()
fig = go.Figure(go.Choropleth( locationmode='geojson-id', geojson=counties, locations=temp_df['GEOID'],
                              z=temp_df['tot_harvest'], colorscale="viridis")
                             )
fig.update_geos(visible=False, scope='usa',
               showsubunits=True,  subunitcolor='black', subunitwidth=1)
fig.show()

fig = go.Figure(go.Choropleth( locationmode='geojson-id', geojson=counties, locations=temp_df['GEOID'],
                              z=temp_df['irri_harvest'], colorscale="viridis")
                             )
fig.update_geos(visible=False, scope='usa',
               showsubunits=True,  subunitcolor='black', subunitwidth=1)

In [15]:
## calculation of yearly irrigated harvest area percent
print('total_geoid: ', harvest['GEOID'].nunique())
harvest['yr_irri_frac'] = harvest['irri_harvest'] / harvest['tot_harvest']

print('any null geoid shape?', harvest.loc[harvest['yr_irri_frac'].isnull()].shape,'any null geoid number', harvest.loc[harvest['yr_irri_frac'].isnull()]['GEOID'].nunique())

## remove null counties:
harvest = harvest.loc[~harvest['yr_irri_frac'].isnull()]
print('After removing null counties, Shape: ', harvest.shape, 
      'GEOID #: ', harvest.loc[~harvest['yr_irri_frac'].isnull()]['GEOID'].nunique())

total_geoid:  2625
any null geoid shape? (6660, 11) any null geoid number 1886
After removing null counties, Shape:  (4969, 11) GEOID #:  1492


In [17]:
## calculate the averaged irrigated harvest area fraction:
### sd will be np.nan for GEOID which has one year data.
harvest['avg_yr_irri_frac']=harvest.groupby(['GEOID'])['yr_irri_frac'].transform('mean')
harvest['yr_irri_frac_sd']=harvest.groupby(['GEOID'])['yr_irri_frac'].transform('std')
harvest.loc[harvest['yr_irri_frac_sd'].isnull()]['GEOID'].nunique()

385

In [20]:
## double-check: 1 year data - null values for SD:
harvest.groupby(['GEOID'])['irri_harvest'].count().reset_index().groupby(['irri_harvest'])['GEOID'].count()

irri_harvest
1    385
2    184
3    109
4    181
5    633
Name: GEOID, dtype: int64

In [24]:
## counties having only one year data:
print('any counties which have irrigation more than 0.1 and one year data:')
print(harvest.loc[(harvest['avg_yr_irri_frac']> 0.1) & (harvest['yr_irri_frac_sd'].isnull())][['GEOID','year']].drop_duplicates().groupby('year')['GEOID'].count())

any counties which have irrigation more than 0.1 and one year data:
year
1997.0    17
2002.0     9
2007.0    29
2012.0    51
2017.0    38
Name: GEOID, dtype: int64


In [22]:
## apply > 10% to the averaged irrigated harvest fraction:
harvest.loc[harvest['avg_yr_irri_frac'] > 0.1, 'irri_counties'] = 'Yes'
print('# of GEOID > 0.1 irrigation:', harvest.loc[harvest['irri_counties'] == 'Yes']['GEOID'].nunique())


# of GEOID > 0.1 irrigation: 797


,State,state_id,county_id,GEOID,year,tot_harvest,irri_harvest,tot+irri_har_merge,lon,lat,yr_irri_frac,avg_yr_irri_frac,yr_irri_frac_sd
10,ALABAMA,01,047,01047,2017.0,13.876,4.111,both,-87.108675,32.326866,0.296267,0.219562,0.108477
12,ALABAMA,01,047,01047,2007.0,5.670,0.810,both,-87.108675,32.326866,0.142857,0.219562,0.108477
15,ALABAMA,01,051,01051,2017.0,2.487,1.100,both,-86.145190,32.600026,0.442300,0.294010,0.120840
16,ALABAMA,01,051,01051,2012.0,2.517,0.691,both,-86.145190,32.600026,0.274533,0.294010,0.120840
17,ALABAMA,01,051,01051,2007.0,2.176,0.676,both,-86.145190,32.600026,0.310662,0.294010,0.120840


In [25]:
## merging the irrigation county indicator to usda_crop data:
usda_data['irri_counties'] = 'No'
usda_data.loc[usda_data['GEOID'].isin(harvest.loc[harvest['irri_counties'] == 'Yes']['GEOID'].unique()),'irri_counties'] = 'Yes'
print(usda_data.loc[usda_data['irri_counties'] == 'Yes']['GEOID'].nunique())

767


In [ ]:
## saving data:
usda_data.to_csv(data_path+'usda_'+crop_yld_name+'_1981_2020_with_irrigation_indicator_3June2022.csv')